The follwoing code does classifies english sentences as erroneous or not, grammar wise. <br /> 
Using 5% of the colnll14 data: 100 correct + 100 erroneous sentences, because using 8000 sentences or 20% of the data 
was causing the kernel's death. <br /> 
Furthermore, two models are used to achieve this classification task: a Random Forrest, and a Multi-Layer Perceptron and that's for the sake of comparaison. <br /> 
The final conclusion is that the random forrest model outperforms the neural network, which is consistent with the results found by the CS229 collaborators. 

In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files

## Prepare the data

The following are two Unix-based commands used to prepare the Colnn14 data to fit the structure requird by 'load_files'

In [ ]:
# Separate the files ‘neg’ & ‘pos’ into text files, each containing one sentence. 
# To be used by the ‘load_files’ function of Sci-Kit Learn

split -d -l 1 neg se --additional-suffix=.txt
split -d -l 1 pos se --additional-suffix=.txt

In [ ]:
# Pick the percentage of the data that you want to use by copying a set number of sentences
k=1; find /Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/txt_sentoken_conll14/pos -type f | while read file; do
     [[ k++ -le 100 ]] && cp "$file" /Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/esmtxt_sentoken_conll14/esmpos 
done

k=1; find /Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/txt_sentoken_conll14/neg -type f | while read file; do
     [[ k++ -le 100 ]] && cp "$file" /Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/esmtxt_sentoken_conll14/neg 
done

Load the data from your system

In [2]:
# Pass the path to the folder containing the 'neg' & 'pos' subfolders.
# Check the link to get a better understanding of how the magical function 'load_files' functions
nucle = load_files(r"/Users/highsierra/Tech-Skills/Labortory/ML-Fundamentals/esmtxt_sentoken_conll14")
X, y = nucle.data, nucle.target

Link: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html

In [3]:
print(X[1:3])
print(type(X))

[b'One example is the recent government project of the integrated resort , which will create job opportunities and attract people to spend money when it is completed .\n', b'Due to the negative consequences of allowing individuals to make the about whether or not to reveal their genetic test results , they should therefore not be given the right to have the final say in the matter .\n']
<class 'list'>


## Clean the data

In [4]:
documents = []

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r"[^a-zA-Z/'/]", ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b' and ending with 'n'
    document = re.sub(r"^b\'", '', document)
    document = re.sub(r"^b", '', document)
    document = re.sub(r"n\'", '', document)

    documents.append(document)

In [5]:
print(len(documents))

200


In [6]:
le=[]
lis=[]
li=[]

for t in documents:
    le = t.split()
    lis.append(le)
    
for e in lis:
    while len(e)<25:
        e.append('space')

texts=[]
for e in lis:
    str = " ".join(e)
    texts.append(str)
print(texts[1])

One example is the recent government project of the integrated resort which will create job opportunities and attract people to spend money when it is completed


## Create POS features from the data

In [7]:
import spacy

nlp = spacy.load("en_core_web_sm")
l1=[]
l2=[]

for text in texts:
        doc = nlp(text)
        l1 = list([token.text, token.pos_, token.tag_, token.dep_, token.shape_] for token in doc)
        l2.extend(l1)
        arr = np.array(l2)

print(arr.shape)

(5535, 5)


## Encode features using One Hot Encoding

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [0, 1, 2, 3, 4])], remainder='passthrough')

arr = transformer.fit_transform(arr)
print(arr.toarray())
print(arr.toarray().shape)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
(5535, 1077)


## Reshape the features matrix to be of same rows as sentences' matrix

In [9]:
print(arr.toarray().dtype)
print(np.sum(arr))

float64
27675.0


In [10]:
arr2d= np.reshape(arr, (5, -1))
print(arr2d.toarray())
print(arr2d.shape)
print(np.sum(arr2d))

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]
(5, 1192239)
27675.0


In [11]:
wall = np.zeros((5,1192240),dtype=np.float64)

In [12]:
print(wall)
print(np.sum(wall))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0.0


In [13]:
wall[:, 0:1192239] += arr2d

In [14]:
print(wall.shape)
print(np.sum(wall))

(5, 1192240)
27675.0


In [15]:
wall2d= np.reshape(wall, (200,-1))

In [16]:
print(wall2d.shape)
print(np.sum(wall2d))

(200, 29806)
27675.0


## Encode sentences using 25-dim glove embeddings

In [17]:
from zeugma.embeddings import EmbeddingTransformer

glove = EmbeddingTransformer('glove') 
X = glove.transform(documents)

Using TensorFlow backend.


In [18]:
print(X.shape)
print(X[3])

(200, 25)
[-1.30604401e-01  2.19229817e-01 -5.71644418e-02  1.40366048e-01
 -2.00404957e-01 -2.34635159e-01  1.18768394e+00 -4.88223344e-01
 -1.05908990e-01 -3.86272520e-02 -1.35389075e-01  4.71128464e-01
 -4.59327841e+00  1.00723945e-01 -2.38876436e-02 -3.94855393e-03
  2.80245155e-01 -9.55506191e-02  8.85953382e-02 -4.58416194e-01
  8.56998265e-02  4.70229834e-01 -7.97560960e-02 -6.56158403e-02
 -3.08540136e-01]


## Combine features and sentences into one input matrix

In [19]:
print("concatenate column wise")
concat = np.concatenate((X,wall2d),axis=1)
print(concat)

concatenate column wise
[[-0.00557672  0.12866175 -0.28674632 ...  0.          0.
   0.        ]
 [-0.0459268   0.33395836 -0.33841449 ...  0.          0.
   0.        ]
 [ 0.28554589  0.34604859 -0.27926505 ...  0.          0.
   0.        ]
 ...
 [ 0.1303066   0.10723669 -0.47099614 ...  0.          0.
   0.        ]
 [ 0.42725652  0.42168248 -0.18633699 ...  0.          0.
   0.        ]
 [ 0.05842838  0.136352   -0.11073975 ...  0.          0.
   0.        ]]


#### Final dimensions verification:

In [20]:
print(y.shape)
print(y)

(200,)
[0 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1
 0 0 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 0 1 1
 1 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1
 0 1 1 1 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1
 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1]


In [21]:
print(concat.shape)

(200, 29831)


## Create training & testing sets

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(concat, y, test_size=0.2, random_state=0)

## Train the Random Forrest model

In [24]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_features=20, n_estimators=200, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=20,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Make predictions

In [25]:
y_pred = classifier.predict(X_test)

## Evaluate the model

In [26]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 4 18]
 [ 3 15]]
              precision    recall  f1-score   support

           0       0.57      0.18      0.28        22
           1       0.45      0.83      0.59        18

    accuracy                           0.48        40
   macro avg       0.51      0.51      0.43        40
weighted avg       0.52      0.47      0.42        40

0.475


## Train the FFNN model

In [27]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(300,200,200,100), max_iter=500, activation = 'relu', solver='adam', random_state=21, tol=0.000000001)
clf.fit(X_train, y_train) 

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300, 200, 200, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=21, shuffle=True, solver='adam',
              tol=1e-09, validation_fraction=0.1, verbose=False,
              warm_start=False)

## Make predictions

In [28]:
y_pred = clf.predict(X_test)

## Evaluate the model

In [29]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 9 13]
 [ 9  9]]
              precision    recall  f1-score   support

           0       0.50      0.41      0.45        22
           1       0.41      0.50      0.45        18

    accuracy                           0.45        40
   macro avg       0.45      0.45      0.45        40
weighted avg       0.46      0.45      0.45        40

0.45
